In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import datetime as dt
import io

In [2]:
df = pd.read_csv('ed_icu_intime_latest.csv')
df.head()

,encounter_id,age,gender,sirs,cci,SOFA_icu_admision,sbp,dbp,mbp,temperature,...,hospital_death_flag,los_hospital,los_icu,ed_intime,ed_outtime,admission_location,icu_intime,icu_outtime,icustay_seq,sepsis3_diagnosis_time
0,34332447,94.0,1.0,3,6.0,6.0,120.00,100.0,113.333333,36.222222,...,0.0,4,1.21,2134-06-18 09:45:00,2134-06-18 13:25:00,EMERGENCY ROOM,2134-06-18 13:25:00,2134-06-19 18:08:21,1,2134-06-18 14:00:00
1,31981707,94.0,0.0,0,9.0,9.0,137.00,69.0,114.333333,37.000000,...,0.0,29,11.00,2170-04-06 16:19:00,2170-04-06 20:07:00,EMERGENCY ROOM,2170-04-06 20:07:00,2170-04-17 20:28:50,1,2170-04-06 21:00:00
2,36935008,93.0,0.0,4,5.0,4.0,120.00,74.0,104.666667,39.833333,...,0.0,10,1.17,2152-07-24 09:04:00,2152-07-24 09:57:02,EMERGENCY ROOM,2152-07-24 09:57:02,2152-07-25 13:46:46,1,2152-07-24 12:00:00
3,30492683,93.0,1.0,4,7.0,7.0,137.61,76.5,117.240000,38.152778,...,0.0,6,2.21,2202-12-21 08:01:00,2202-12-21 12:37:00,EMERGENCY ROOM,2202-12-21 12:37:00,2202-12-23 17:56:47,1,2202-12-22 06:00:00
4,39334847,94.0,1.0,1,8.0,2.0,133.00,83.0,116.333333,35.000000,...,0.0,10,1.08,2195-05-29 17:56:00,2195-05-29 21:51:00,EMERGENCY ROOM,2195-05-29 21:51:00,2195-05-30 23:00:21,1,2195-05-30 02:00:00


In [3]:
continous = ['age', 'sirs']
categorical = ['gender', 'hospital_death_flag']

In [4]:
def makeContinuous(dataframe, continuous):
    #make null list
    median_list = []
    iqr_list = []
    
    #for each column
    for column in continuous:
        #calculate each qx
        dataframe[column] = dataframe[column].astype(float)
        q1 = dataframe[column].quantile(0.25).astype(float)
        q2 = dataframe[column].quantile(0.50).astype(float)
        q3 = dataframe[column].quantile(0.75).astype(float)
        #IQR
        iqr = f'[{q1}-{q3}]'
        #appen to list
        median_list.append(q2)
        iqr_list.append(iqr)
        
    #make df
    df_continuous = pd.DataFrame(list(zip(continuous, median_list, iqr_list,)), columns=['variables', 'main', 'sub',])
   
    return df_continuous

In [12]:
def makeContinuous(dataframe, continuous):
    
    df_continuous = pd.DataFrame(columns=continuous, index=['main', 'sub'])
    
    for column in continuous:
        #calculate each qx
        dataframe[column] = dataframe[column].astype(float)
        q1 = dataframe[column].quantile(0.25).astype(float)
        q2 = dataframe[column].quantile(0.50).astype(float)
        q3 = dataframe[column].quantile(0.75).astype(float)
        #IQR
        iqr = f'[{q1}-{q3}]'
        #appen to list
        df_continuous[column].loc['main'] = q2
        df_continuous[column].loc['sub'] = iqr
        
    return df_continuous

In [13]:
df_continuous = makeContinuous(df, continous)

In [8]:
df['a'] = df['gender'].map(lambda x: True if x==1 else False)

In [10]:
df['a'].dtype == bool

True

In [ ]:
def makeCategorical(dataframe, categorical):
    
    df_categorical = pd.DataFrame(columns=categorical, index=['main', 'sub'])
    
    for column in categorical:
        if df[column].dtype == bool:
            df[column] = df[column].map(lambda x:1 if x ==True else False)
        
        n = len(df[df[column]==1])
        rate = round(n/df *100, 1)
        
        #appen to list
        df_categorical[column].loc['main'] = n
        df_categorical[column].loc['sub'] = rate
    
    return df_categorical
    # #order dict
    
    df_continuous = pd.DataFrame(list(zip(continuous, median_list, iqr_list,)), columns=['variables', 'main', 'sub',])